# Tuning Script

In [1]:
# Parameters subject to change in every script - involved in naming

MODEL = 'cb'
TUNER = 'jiaocheng'

---

In [2]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

Mounted at /content/drive


In [3]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    home_directory = './drive/MyDrive/LAB/AFLBrownlowPredictor2024/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebr':
        !pip install interpret==0.5.0
    elif MODEL == 'cb':
        !pip install catboost
    !pip install JXAutoML
else:
    home_directory = '../../'

from JXAutoML.JiaoCheng import JiaoCheng as Tuner

import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.5 MB/s eta 0:00:00


In [4]:
# Parameters that are same for every script
SEED = 42

TASK_TYPE = 'Regression'  # this task is a classification task

# we have decided to tune features as hp
TUNE_FEATURES_AS_HYPERPARAMETERS = True

---

In [5]:
# model to use to tune
from catboost import CatBoostClassifier as clf

# what values to try for each hyperparameter
parameter_choices = {

    'n_estimators': (25, 50, 100, 200),
    'max_depth': (2, 4, 8),
    'subsample': (0.4, 0.55, 0.7, 0.85, 1),
    'colsample_bylevel': (0.4, 0.55, 0.7, 0.85, 1),
    'max_bin': (64, 256),
    'reg_lambda': (0.01, 0.1, 1),
    'learning_rate': (0.001, 0.01, 0.1, 1),

}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
    'random_state': 19260817,
    'verbose': False,
    'bootstrap_type': 'Bernoulli',
}

# what order to tune hyperparameters in (JiaoCheng specific)
tuning_order = ['features',
                'max_depth',
                'max_bin',
                'colsample_bylevel',
                'subsample',
                'n_estimators',
                'reg_lambda',
                'learning_rate']

# what order to tune hyperparameters in (JiaoCheng specific)
default_hyperparameter_values = {
    'features': 0,
    'n_estimators': 50,
    'max_depth': 4,
    'subsample': 0.7,
    'colsample_bylevel': 0.7,
    'learning_rate': 0.01,
    'reg_lambda': 0.01,
    'max_bin': 64,
}

---

Read in and Prepare Data

In [6]:
train_data = pd.read_csv(f'{home_directory}data/curated/modelling/train.csv')
val_data = pd.read_csv(f'{home_directory}data/curated/modelling/val.csv')
test_data = pd.read_csv(f'{home_directory}data/curated/modelling/test.csv')

train_X = train_data.drop(columns=['target'])
train_y = train_data['target']
val_X = val_data.drop(columns=['target'])
val_y = val_data['target']
test_X = test_data.drop(columns=['target'])
test_y = test_data['target']

In [7]:
# Import feature importance ordering

with open(f'{home_directory}models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [8]:
# initialisation
tuner = Tuner()

print('---')

# define what model we are tuning
tuner.read_in_model(clf, TASK_TYPE, pytorch_model=False, optimised_metric='r2')

print('---')

# read in the data for training and validation
tuner.read_in_data(train_X, train_y, val_X, val_y, test_X, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
    tuner.set_features(feature_importance_ordering)
    # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up the order of hyperparameters when iteratively tuning using JiaoCheng
tuner.set_tuning_order(tuning_order)

print('---')

# set up the default hp values for first iteration of tuning JiaoCheng
tuner.set_hyperparameter_default_values(default_hyperparameter_values)

print('---')

try:  # try to read in previous results to continue tuning ...
    tuner.read_in_tuning_result_df(
        f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')
except:
    print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(
    f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')

print('---')

# set up where to save the current best model
tuner.set_best_model_saving_address(
    f'{home_directory}models/tmp_models/{TUNER}_{MODEL}')

JiaoCheng Initialised
---
Successfully read in model <class 'catboost.core.CatBoostClassifier'>, which is a Regression model optimising for r2
---
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
---
Successfully recorded hyperparameter choices
---
Successfully recorded non_tuneable_hyperparameter choices
---
Successfully recorded tuneable feature combination choices and updated relevant internal structures
---
---
---
Error message: ('Effective Disposals', 'Disposals', 'Contested Possessions', 'Goals', 'Hitouts')
Error Importing this Row: (4, n_estimators                                                                  50.0
max_depth                                                                      4.0
subsample                                                                      0.7
colsample_bylevel                                                              0.7
max_bin                                        

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
# begin tuning ... (or continue training)

tuner.tune()


Default combo: [1, 1, 2, 2, 0, 0, 1, 0] 


ROUND 1

Round 1 
Hyperparameter: features (index: 7) 

As new Best Combo (1, 1, 2, 2, 0, 0, 1, 0) was read in, best_clf is set to None
Already Trained and Tested combination (1, 1, 2, 2, 0, 0, 1, 0), which had val score of 0.3723
        Current best combo: (1, 1, 2, 2, 0, 0, 1, 0) with val score 0.3723. 
        Has trained 152 of 266400 combinations so far
As new Best Combo (1, 1, 2, 2, 0, 0, 1, 1) was read in, best_clf is set to None
Already Trained and Tested combination (1, 1, 2, 2, 0, 0, 1, 1), which had val score of 0.4273
        Current best combo: (1, 1, 2, 2, 0, 0, 1, 1) with val score 0.4273. 
        Has trained 152 of 266400 combinations so far
As new Best Combo (1, 1, 2, 2, 0, 0, 1, 2) was read in, best_clf is set to None
Already Trained and Tested combination (1, 1, 2, 2, 0, 0, 1, 2), which had val score of 0.5059
        Current best combo: (1, 1, 2, 2, 0, 0, 1, 2) with val score 0.5059. 
        Has trained 152 of 266400 co